In [1]:
import cobra
import pandas as pd
import os

# -----------------------------
# 1. Load the metabolic model
# -----------------------------
model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"SBML model not found at {model_path}")

model = cobra.io.read_sbml_model(model_path)
model.objective = "BIOMASS_Ec_SynAuto_1"

# -----------------------------
# 2. Calculate WT fluxes
# -----------------------------
wt_solution = model.optimize()
wt_fluxes = wt_solution.fluxes
wt_fluxes.name = "WT_flux"

# -----------------------------
# 3. Run single-gene knockouts and store fluxes
# -----------------------------
gene_list = [g.id for g in model.genes]
flux_records = []

# For demonstration, limit to first 20 genes; remove [:20] to do all genes
for gene in gene_list[:20]:
    with model:
        model.genes.get_by_id(gene).knock_out()
        ko_solution = model.optimize()
        flux = ko_solution.fluxes
        flux_records.append(pd.DataFrame({
            "reaction_id": flux.index,
            "KO_flux": flux.values,
            "gene_knocked_out": gene
        }))

# Combine all KO fluxes into one DataFrame
ko_flux_df = pd.concat(flux_records, ignore_index=True)

# -----------------------------
# 4. Merge WT fluxes for comparison
# -----------------------------
wt_df = wt_fluxes.reset_index().rename(columns={"index": "reaction_id", "WT_flux": "WT_flux"})
flux_comparison = ko_flux_df.merge(wt_df, on="reaction_id")
flux_comparison["flux_change"] = flux_comparison["KO_flux"] - flux_comparison["WT_flux"]
flux_comparison["percent_change"] = flux_comparison["flux_change"] / flux_comparison["WT_flux"].replace(0, 1e-9) * 100

# -----------------------------
# 5. Identify key metabolites impacted
# -----------------------------
# Sum absolute flux changes per metabolite
metabolite_changes = {}

for rxn_id, change in zip(flux_comparison["reaction_id"], flux_comparison["flux_change"]):
    rxn = model.reactions.get_by_id(rxn_id)
    for met in rxn.metabolites:
        if met.id not in metabolite_changes:
            metabolite_changes[met.id] = 0
        metabolite_changes[met.id] += abs(change)

metabolite_impact_df = pd.DataFrame.from_dict(metabolite_changes, orient="index", columns=["total_flux_change"])
metabolite_impact_df = metabolite_impact_df.sort_values("total_flux_change", ascending=False)

# -----------------------------
# 6. Save summary statistics
# -----------------------------
# Summary per reaction
reaction_summary_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results\flux_summary_per_reaction.csv"
flux_comparison.to_csv(reaction_summary_path, index=False)
print(f"✅ Reaction flux summary saved: {reaction_summary_path}")

# Summary per metabolite
metabolite_summary_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict\ProkaryPredict-McCrory\results\metabolite_flux_impact.csv"
metabolite_impact_df.to_csv(metabolite_summary_path)
print(f"✅ Metabolite impact summary saved: {metabolite_summary_path}")

# Preview top impacted metabolites
metabolite_impact_df.head(10)

No objective coefficients in model. Unclear what should be optimized


✅ Reaction flux summary saved: C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results\flux_summary_per_reaction.csv


OSError: Cannot save file into a non-existent directory: 'C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict\ProkaryPredict-McCrory\results'